In [ ]:
print("Loading Libraries...")
import nltk
nltk.download('punkt')
from nltk.stem import PorterStemmer
stemmer = PorterStemmer()
import tensorflow as tf
import numpy as np
import random
import json
print("Loading the Dataset...")
with open('tamil_intents_transports.json') as json_data:
    intents = json.load(json_data)
words = []
classes = []
documents = []
ignore = ['!']
print("Data Preprocessing...")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        documents.append((w, intent['tag']))
        if intent['tag'] not in classes:
            classes.append(intent['tag'])
words = [stemmer.stem(w.lower()) for w in words if w not in ignore]
words = sorted(list(set(words)))
classes = sorted(list(set(classes)))
training = []
output = []
output_empty = [0] * len(classes)
print("Training the Model...")
for doc in documents:
    bag = []
    pattern_words = doc[0]
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    training.append([bag, output_row])
random.shuffle(training)
training = np.array(training, dtype = object)
train_x = list(training[:,0])
train_y = list(training[:,1])
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(10,input_shape=(len(train_x[0]),)))
model.add(tf.keras.layers.Dense(10))
model.add(tf.keras.layers.Dense(len(train_y[0]), activation='softmax'))
model.compile(tf.keras.optimizers.Adam(), loss='categorical_crossentropy', metrics=['accuracy']) 
model.fit(np.array(train_x), np.array(train_y), epochs=1000, batch_size=10, verbose=0)
model.save("model.pkl")
import pickle
pickle.dump( {'words':words, 'classes':classes}, open( "training_data", "wb" ) )
print("Model Created Successfully...")